In [1]:
%load_ext autoreload

%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from kPOD import k_pod

In [4]:
path = 'sm_data.nc'
ds = xr.open_dataset(path)

In [5]:
data = ds.transpose()

In [6]:
def generate_test_train(data, boxes=39, sensors=6, t1=80000, t2=150000, v1=150000, v2=160000):
    train_features = []
    test_features = []
    train_labels = []
    test_labels = []
    for i in range(boxes):
        for j in range(sensors):
            train_feature = pd.Series(data['moisture'][j][i][t1:t2])
            train_label = pd.Series(data['moisture_flag'][j][i][t1:t2])
            train_frame = { 'Feature': train_feature, 'Label': train_label }
            train_df = pd.DataFrame(train_frame)
            train_df.loc[train_df['Label'] != 'OK', 'Feature'] = np.nan
            train_features.append(np.array(train_df['Feature']))
            train_labels.append(np.array(train_df['Label']))
            test_feature = pd.Series(data['moisture'][j][i][v1:v2])
            test_label = pd.Series(data['moisture_flag'][j][i][v1:v2])
            test_frame = { 'Feature': test_feature, 'Label': test_label }
            test_df = pd.DataFrame(test_frame)
            test_df.loc[test_df['Label'].isin(['Auto:Range','Auto:Spike', 'Auto:BattV']), 'Feature'] = np.nan
            test_features.append(np.array(test_df['Feature']))
            test_labels.append(np.array(test_df['Label']))
    return train_features, train_labels, test_features, test_labels

In [11]:
""" Data input as feature set e.g. """
def cov_matrix(data1):
    m = []
    for i in range(len(data1)):
        row = []
        for j in range(len(data1)):
            if i == j:
                row.append(0)
            else:
                row.append(pd.Series(data1[i]).cov(pd.Series(data1[j])))
        m.append(row)
    maximum = max([abs(min(np.concatenate(m))), abs(max(np.concatenate(m)))])
    m = m / (maximum if maximum != 0 else 1)
    return m

In [8]:
train_fs, train_ls, test_fs, test_ls = generate_test_train(data)

In [7]:
k_size = 8

In [13]:
clustering_results = k_pod(train_fs, k_size)

In [10]:
def cluster_lookup(clustering_results1):
    cluster_lookup = {}
    for i in range(len(clustering_results1[0])):
        index = int(clustering_results1[0][i])
        if index not in cluster_lookup.keys():
            cluster_lookup[index] = []
        cluster_lookup[index].append(i)
    return cluster_lookup

In [14]:
cl = cluster_lookup(clustering_results)